        stty -F /dev/ttyACM* 57600 ixon
        sleep 4
        stty cols 53 rows 20
        price | tee /dev/ttyACM*

# ANSI serial terminal test

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [19]:
BAUDRATE = 57600
#BAUDRATE = 38400
CW,CH = 6,12
TERMROWS = 20
TERMCOLS = 53

pauseaftershow = 5 #seconds

CONTROL            = '\x1b'
CSI                = CONTROL+'['
clearafter         = CSI + '0J'
clearbefore        = CSI + '1J'
clearscreen        = CSI + '2J'
clearscreen2       = CSI + '3J'
clearright         = CSI + '0K'
clearleft          = CSI + '1K'
clearline          = CSI + '2K'
underline          = CSI + '3m'
underline          = CSI + '4m'
double_underline_1 = CSI + '21m'
double_underline_2 = CSI + '61m'
overline_1         = CSI + '53m'
overline_2         = CSI + '62m'
double_overline    = CSI + '63m'
strike             = CSI + '9m'
verybold           = CSI + '20m'
vtab               = '\x0b'
upstart            = CSI+'F'
downstart          = CSI+'E'
savecursor         = '\x1b7'
restorecursor      = '\x1b8'
showcursor         = CSI+"?25h"
hidecursor         = CSI+"?25l"
reset              = '\x1bc'
getcursor          = CSI+'6n'

def scroll(n):
    return CSI+'%d'%abs(n)+('T' if n<0 else 'S')

def goto(r,c):
    return '\x1b[%d;%dH'%(r+1,c+1)

up    = lambda n: '\x1b[%dA'%n
down  = lambda n: '\x1b[%dB'%n
left  = lambda n: '\x1b[%dD'%n
right = lambda n: '\x1b[%dC'%n

def coljump(c):
    return CSI+'%d'%(c+1)+'G'

In [20]:
import os,serial,time,sty
from sty import fg, bg

ports = [f for f in os.listdir('/dev/') if 'acm' in f.lower() or 'ttyusb' in f.lower()]
print('Available ports:\n  '+'\n  '.join(ports))
port = '/dev/'+ports[0]
print('Using port',port)
arduino = serial.Serial(port=port, 
                        baudrate=BAUDRATE, 
                        xonxoff=False,  
                        timeout=.1) 
                        #write_timeout=5)

import os
#os.system('stty -F %s %s ixon cr3 ff1 nl1 bs1'%(port,BAUDRATE))
time.sleep(3)

def print2(s='',delay=None):
    if delay == None: 
        delay=0
    throttle = delay*(len(s)/TERMCOLS+s.count('\n'))
    delay = throttle / len(s)
    if delay>0: print(1/delay*8)
    for b in s:
        arduino.write(b.encode('utf-8'))
        arduino.flush()
        time.sleep(delay)
        
def print_arduino(s):
    arduino.write(s.encode('utf-8'))

def println2(s='',delay=None):
    print2(s+'\n',delay=delay)


Available ports:
  ttyACM0
Using port /dev/ttyACM0


In [22]:

print2(sty.rs.all + clearscreen)
print2(goto(6,55//2-8)+fg(255,100,0)+'◖'+sty.ef.inverse+'Hello world!'+sty.rs.inverse+'◗')
print2(goto(8,55//2-12)+fg(255,100,0)+'◖'+sty.ef.inverse+'This is an Arduino TTY'+sty.rs.inverse+'◗')
print2(goto(10,0)+'Its running at a baud rate of %d, and expects utf-8 encoded text. ANSI styling commands are supported. '%BAUDRATE)
time.sleep(pauseaftershow)

# Positioning, clearing, scrolling

In [23]:
ANIMATION_DELAY = 0.35

print2(sty.rs.all + clearscreen)

# Show/hide the cursor
println2(showcursor+"[?25h shows the cursor")
time.sleep(ANIMATION_DELAY)

println2(hidecursor+"[?25l hides it")
time.sleep(ANIMATION_DELAY)

println2(CSI+"?25hlet's show it for now")
time.sleep(ANIMATION_DELAY)

print2(bg(7) + clearscreen + 'clear screen ]2J gray')
time.sleep(ANIMATION_DELAY)

print2(bg.green + clearscreen2 + 'clear screen ]3J green')
time.sleep(ANIMATION_DELAY)

print2(goto(TERMROWS//2-1,TERMCOLS//2))
print2(bg.red  + clearbefore + '\r clear before ]1J red')
time.sleep(ANIMATION_DELAY)

print2(goto(TERMROWS//2-1,TERMCOLS//2))
print2(bg(5)+clearright+'     clear right ]0K mauve')
time.sleep(ANIMATION_DELAY)

print2(goto(TERMROWS//2,TERMCOLS//2))
print2(bg.blue + clearafter + '     clear after ]0J blue')
time.sleep(ANIMATION_DELAY)

print2(goto(TERMROWS//2,TERMCOLS//2))
print2(bg(6)+clearleft+'\r clear left   ]1K cyan')
time.sleep(ANIMATION_DELAY)

print2(bg(3))
print2(scroll(2)  + goto(0,0) + 'Scrolled up   2 [2S brown')
time.sleep(ANIMATION_DELAY)

print2(bg(2))
print2(scroll(-1) + goto(TERMROWS-2,0) + 'Scrolled down 1 [1T green')
time.sleep(ANIMATION_DELAY)

time.sleep(2)

# Colors

In [24]:
offset = 1
s = sty.rs.all+clearscreen
s += goto(0,offset) + '16-color pallet:'
s += goto(1,offset)
for i in range(16):
    s += bg(i)+'  '
s += goto(2,offset) + bg(0) + '6x6x6 color cube:'
for r in range(6):
    s += goto(r+3,offset)
    for g in range(6):
        for b in range(6):
            s += bg(b+g*6+r*36+16)+' '
        s += bg(0)+' '
s += goto(9,offset)+"Grays (can\'t render all with 8-bit RRRBBGGG color)"
s += goto(10,offset)
for q in range(24):
    s+=bg(q+232)+' '
s += goto(11,offset)+sty.rs.all
s += '[*;2;R;G;Bm color codes\n '
for g in range(8):
    for r in range(8):
        for b in range(4):
            s += bg(round(r*255/7),round(g*255/7),round(b*255/3))+' '
    s += goto(13+g,offset)
xoffset = 53-8*2-1
s += goto(10,xoffset+3)+sty.rs.all+'Colors 0-7'
s += goto(11,xoffset)
s += 'Fore/background'
for j in range(8):
    s += goto(12+j,xoffset)
    s += bg(j)
    for i in range(8):
        s += fg(i+8)
        s += '%d%d'%(i,j)
print2(s,delay=0);

time.sleep(3)

## Control / navigation 

In [25]:
print2(sty.rs.all + clearscreen)
time.sleep(ANIMATION_DELAY)
println2('(cursor starts at 0,0)')
time.sleep(ANIMATION_DELAY)
print2('(Save it with control+7)'+bg.red+' '+sty.rs.all+savecursor)
time.sleep(ANIMATION_DELAY)
print2(goto(2,4)+'(... then move somewhere)')
time.sleep(ANIMATION_DELAY)
print2(restorecursor + '(Then return via control+8)')
time.sleep(ANIMATION_DELAY)
print2(goto(3,0))
time.sleep(ANIMATION_DELAY)
print2('carriage return... we\'ll overwrite this in a second')
time.sleep(ANIMATION_DELAY)
println2('\r'+'carr. rtn.: (done)')
time.sleep(ANIMATION_DELAY)
println2('bell  "\\a": \a(did the screen flash?)')
time.sleep(ANIMATION_DELAY)
print2('bksp  "\\b": we\'ll delete this in a second')
time.sleep(ANIMATION_DELAY)
println2('\b'*len("we\'ll delete this in a second") + '(deleted)')
time.sleep(ANIMATION_DELAY)
println2('tab   "\\t": tab\ttab')
time.sleep(ANIMATION_DELAY)
println2('ffeed   12: B\fB')
time.sleep(ANIMATION_DELAY)
println2('vtab    11: A'+vtab+'Hi.')
time.sleep(ANIMATION_DELAY)
print2('navigation test -----  +'+
       left(2)+'L'+
       up(1)  +'U'+
       left(1)+down(2)+'D'+
       left(1)+up(1)+right(1)+'R',delay=1)
time.sleep(ANIMATION_DELAY)
print2(down(3)+downstart+'(down to start of line) ]E')
time.sleep(ANIMATION_DELAY)
print2(upstart+'(up   to start of line) ]F')
time.sleep(ANIMATION_DELAY)
print2(down(1)+downstart+':'+coljump(TERMCOLS//2)+'<- Jump to column')
time.sleep(ANIMATION_DELAY)

# We can also request the cursor position
print2(getcursor)
print((b''.join([arduino.read() for i in range(10)])))


time.sleep(2)

424.0
b'\x00\x00\x13\x11\x1b[000;'


## Text effects

In [26]:
def ul(*args):
    # Set underlying color
    if len(args)==1 and args[0]<8:
        return '\x1b[%dm'%(100+args[0])
    return bg(*args).replace('[48','[58')
sty.ul = ul

color = fg(255,255,255)
fonttest = 'AaBbCcDdEeFfGg1234567890' + sty.rs.all
underline          = '\x1b[3m'
underline          = '\x1b[4m'
double_underline_1 = '\x1b[21m'
double_underline_2 = '\x1b[61m'
overline_1         = '\x1b[53m'
overline_2         = '\x1b[62m'
double_overline    = '\x1b[63m'
strike             = '\x1b[9m'
verybold           = '\x1b[20m'

s = clearscreen + sty.rs.all+fg.white+bg.black 
s += 'Fancy font effects:\n\n'
s += 'Bold    [1: '+sty.ef.bold+fonttest+ '\n'
s += 'Faint   [2: '+sty.ef.dim+fonttest+ '\n'
s += 'Italic  [3: '+sty.ef.italic+fonttest+ '\n'
s += 'Bld+It    : '+sty.ef.bold+sty.ef.italic+fonttest+ '\n'
s += 'Blink   [5: '+sty.ef.blink+fonttest+ '\n'
s += 'BBold  [20: '+verybold+fonttest+ '\n'
s += 'B+BBld    : '+sty.ef.bold+verybold+fonttest+ '\n'
s += 'Invert  [7: '+sty.ef.inverse+fonttest+ '\n'
s += '1xOver [53: '+ul(1)+overline_1+fonttest+ '\n'
s += '1xOver [62: '+ul(1)+overline_2+fonttest+ '\n'
s += '2xOver [63: '+ul(1)+double_overline+fonttest+ '\n'
s += '1xUnder [4: '+ul(1)+underline+fonttest+ '\n'
s += '2xUnder[21: '+ul(1)+double_underline_1+fonttest+ '\n'
s += '2xUnder[61: '+ul(1)+double_underline_2+fonttest+ '\n'
s += 'Strike  [9: '+ul(1)+strike+fonttest+ '\n'
s += '2xO 2xU S : '+ul(1)+double_underline_1+double_overline+strike+fonttest+ '\n'

print2(s,delay=0.005)
time.sleep(6)

39503.69540873461


# Unicode

In [27]:
print2(clearscreen + sty.rs.all+fg.white+bg.black)

s = 'Braille patterns:\n\n'
for i in range(0x2800,0x28FF+1):
    s += chr(i)
    
s += '\n\nBlock sextants:\n\n'
for i in range(0x1FB00,0x1fb3b+1):
    s+=chr(i)
    
s += '\n\nTeletext:\n\n'
s += ''.join([chr(i) for i in range(0x1FB3C,0x1FB3C+44)])
    
print2(s)
time.sleep(pauseaftershow)

In [28]:
s=reset+hidecursor
s+='Box and Block-Drawing Characters 0x2500-0x259F:\n\n'
#"""
s+='─━│┃┄┅┆┇┈┉┊┋┌┍┎┏┐┑┒┓└┕┖┗┘┙┚┛├┝┞┟┠┡┢┣┤┥┦┧┨┩┪┫┬┭┮┯\n'
s+='┰┱┲┳┴┵┶┷┸┹┺┻┼┽┾┿╀╁╂╃╄╅╆╇╈╉╊╋╌╍╎╏═║╒╓╔╕╖╗╘╙╚╛╜╝╞╟\n'
s+='╠╡╢╣╤╥╦╧╨╩╪╫╬╭╮╯╰╱╲╳╴╵╶╷╸╹╺╻╼╽╾╿▀▁▂▃▄▅▆▇█▉▊▋▌▍▎▏\n'
s+='▐░▒▓▔▕▖▗▘▙▚▛▜▝▞▟\n\n'

#"""
s+='   ╱╲   ┌───┐ ┏━━━┓\n'
s+=' ╱╳╳╳╳╲ │▐█▌├┈┨▝ ▖┣╸╺╸╺╍╍┅┅┉┉━┑\n'
s+=' ╲╳╳╳╳╱ └─┬─┘ ┗━━━┛  ╔═══╗  ╒═╧═╕\n'
s+='   ╲╱     ╰─╌╌┄┄┈┈╼━═╣   ╟──┤   ┆\n'
s+='                     ╚═══╝  ╘═══╛\n'
s+='  ▟▙▝▞▞▝▛▞▙▙▝▞▌▀▖▙\n'
s+='  ▌▟▝▘▛▝▜▚▙ ▙▝▀▛▚▖\n'
s+='  ▞▀█▛▜ ▛▚▟▀▝█▗▞▛▛\n'

print2(s)
time.sleep(pauseaftershow)

In [29]:
core_support = (''' !"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]^_`'''
                '''abcdefghijklmnopqrstuvwxyz{|}~ ¡¢£¤¥¦§¨©ª«¬®¯°±²³´µ¶·¸¹º»¼½¾¿'''
                '''ÀÁÂÃÄÅÆÇÈÉÊËÌÍÎÏÐÑÒÓÔÕÖ×ØÙÚÛÜÝÞßàáâãäåæçèéêëìíîïðñòóôõö÷øùúûüýþÿ'''
                '''ĀāĂăĄąĆćĈĉĊċČčĎďĐđĒēĖėĘęĚěĜĝĞğĠġĢģĤĥĦħĪīĮįİıĴĵĶķĹĺĻļĽľŁłŃńŅņŇňŌōŐőŒœ'''
                '''ŔŕŖŗŘřŚśŜŝŞşŠšŢţŤťŪūŬŭŮůŰűŲųŴŵŶŷŸŹźŻżŽžƒȘșȚțˆˇˉ˘˙˛˜˝΄΅ΆΈΉΊΌΎΏΐΑΒΓΔΕΖΗ'''
                '''ΘΙΚΛΜΝΞΟΠΡΣΤΥΦΧΨΩΪΫάέήίΰαβγδεζηθικλμνξοπρςστυφχψωϊϋόύώϳϴЀЁЂЃЄЅІЇЈЉЊЋЌЍЎЏ'''
                '''АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяѐёђѓєѕіїј'''
                '''љњћќѝўџҐґҮӐӑḂḃḊḋḞḟṀṁṖṗṠṡṪṫẀẁẂẃẄẅỲỳ‐‒–—―‘’‚“”„†‡•…‰‹›‼ⁿ₧€№™Ω←↑→↓∆−∙√∞∩≈≠≡≤≥⌐⌠⌡'''
                '''①②③④⑤⑥⑦⑧⑨⓪─━│┃┌┏┐┓└┗┘┛├┝┠┣┤┥┨┫┬┯┰┳┴┷┸┻┼┿╂╋╪╬█░▒■▲▼◊●''')
s=reset
s+='github.com/ehmicky/cross-platform-terminal-characters\n\n'
s+=core_support
print2(s)
time.sleep(pauseaftershow)

In [30]:
print2(reset+clearscreen+hidecursor+\
       "Legacy Computing Characters (some custom additions):\n"+\
       goto(3,20)+(
'\n'+right(10)+"🭵🮀🮁🮁🮁🮁🮁🮁🮁🮁🮁🮁🮁🮁🮁🮁🮁🮁🮁🮁🮁🮁🮁🮁🮁🮁🮀🮼"+\
'\n'+right(10)+"🭵                          🮸"+\
'\n'+right(10)+"🭵     🮲🮳 🯅 🯆 🯇 🯈 🯉         |  {"+sty.ef.inverse+'ok'+sty.rs.inverse+'}'+\
'\n'+right(10)+"🭵                          |"+\
'\n'+right(10)+"🭵  🯁🯂🯃 🮹🮺     🯰🯱 🯲🯳🯴🯵🯶🯷🯸🯹  .     🮰"+\
'\n'+right(10)+"🭵                          |"+\
'\n'+right(10)+"🭵🮵===========$============🮶🮷"+\
'\n'+right(10)+
'\n'+right(10)+"   🯀 🯄 🮻 🮽 🮾 🮿 🮱 🮴 🮯 "+\
'\n'+right(10)+
'\n'+right(10)+"   🮌 🮍 🮎 🮏 🮐 🮑 🮒 \U0001fb93 🮔 🮕 "+\
'\n'+right(10)+
'\n'+right(10)+"   🮖 🮗 🮘 🮙 🮚 🮛 🮜 🮝 🮞 🮟 "
)
# Add ons: 
.replace('=','\U0001fbcb') # Empty horizontal scroll bar
.replace('$','\U0001fbcc') # Horizontal scroll handle
.replace('.','\U0001fbcd') # Vertical scroll handle
.replace('|','\U0001fbce') # Empty vertical scroll bar
.replace('{','\U0001fbcf') # Left round button end
.replace('}','\U0001fbd0') # Right round button end
)
time.sleep(pauseaftershow)